In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

24/11/23 00:11:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 00:11:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/23 00:11:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/23 00:11:22 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DateType,TimestampType
from pyspark.sql.functions import col, concat,current_timestamp,lit

In [3]:
name_schema = StructType([StructField('forename', StringType()),
                          StructField('surname', StringType())])

In [4]:
driver_schema = StructType([
            StructField('driverId',IntegerType()),
            StructField('driverRef',StringType()),
            StructField('number', IntegerType()),
            StructField('code', StringType()),
            StructField('name',name_schema),
            StructField('dob', DateType()),
            StructField('nationality',StringType()),
            StructField('url',StringType())
])

In [5]:
driver_df = spark.read.schema(driver_schema).json('source/drivers.json')


In [7]:
driver_wiithcolumnrenamed = driver_df.withColumnRenamed('driverId', 'driver_id')\
.withColumnRenamed('driverRef','driver_ref')\
.withColumn('ingestion_date',current_timestamp())\
.withColumn('name',concat(col('name.forename'),lit(' '),col('name.surname'))).drop('url')

In [11]:
driver_wiithcolumnrenamed.write.option('header',True).mode('overwrite').format('csv').save('tbls/drivers')


In [6]:
df = spark.read.option('header',True).option('InferSchema',True).csv('tbls/drivers/drivers.csv')

In [28]:
outs =[]
for i in df.select('nationality').collect():
    outs.append(len(i[0]))
print(max(outs))

17


In [23]:
df.printSchema()

root
 |-- driver_id: integer (nullable = true)
 |-- driver_ref: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)

